### Summary
In this note, we study how to generate random numbers using different distributions.

Basically, there are **two ways** to generate random numbers:
1. use *pytorch functions or methods* like `torch.Tensor().uniform_()`, `torch.bernoulli()`, and `torch.multinomial()`.

    This approach is more straightforward to use, but it is not so flexible and powerful as the 2nd approach below. Docments for some mostly used distribution is listed below
    * Uniform distribution in $[a,b]$, `torch.Tensor(input).uniform_(a,b)` see <a href="https://pytorch.org/docs/1.8.1/tensors.html?highlight=uniform#torch.Tensor.uniform_">here</a> for reference
    * Bernoulli distribution `torch.bernoulli(input)` with probabilities specifed in the *input* tensor argument, see <a href="https://pytorch.org/docs/1.8.1/generated/torch.bernoulli.html#torch-bernoulli">here</a> for the reference
    * Multinomial distribution `torch.multinomial(input)` with probability (or weights) specified by the *input* tensor, see <a href="https://pytorch.org/docs/1.8.1/generated/torch.multinomial.html#torch-multinomial">here</a> for the reference
    * Normal distribution `torch.normal(mean,std)`, see <a href="https://pytorch.org/docs/1.8.1/generated/torch.normal.html#torch-normal">here</a> for the reference


2. use *pytorch class* like `torch.distributions.binomial.Binomial` to generate a desired distribution first, e.g., `my_distribution`, then use *method* like `my_distribution.sample()` to get the random numbers out of the generated distribution. 

    This approach allows for more customization for the desired distribution from which to generate the random numbers. The referenes and examples for some mostly used distrubtions are listed below
    * <a href="https://pytorch.org/docs/1.8.1/distributions.html#torch.distributions.bernoulli.Bernoulli"> Bernoulli distribution </a> 
    * <a href="https://pytorch.org/docs/1.8.1/distributions.html#torch.distributions.binomial.Binomial"> Binomial distribution </a>
    * <a href="https://pytorch.org/docs/1.8.1/distributions.html#torch.distributions.multinomial.Multinomial"> Multinomial distribution </a>
    * <a href="https://pytorch.org/docs/1.8.1/distributions.html#torch.distributions.normal.Normal"> Normal distribution </a> and <a href="https://pytorch.org/docs/1.8.1/distributions.html#torch.distributions.multinomial.Multinomial"> multivariate normal distribution </a>
    
**In this note, we use the first approach listed above.**

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

##### Seed
Firstly, let's make our own seed for random number generator, as is in the Matlab. So that we can fix the batches of random numbers to use.

In [2]:
myseed = 666
myseed

666

In [3]:
torch.manual_seed(myseed)
torch.randn(4)

tensor([-2.1188,  0.0635, -1.4555, -0.0126])

##### Normal distrubtion
Now we use the seed to generate tensors using standard normal distrubtion.

In [4]:
torch.manual_seed(myseed)
rvn1 = torch.randn(4,4) # generate a 4-by-4 tensor with standard normal distribution 
rvn1

tensor([[-0.7747,  0.7926, -0.0062, -0.4377],
        [ 0.4657, -0.1880, -0.8975,  0.4169],
        [-0.3840,  0.0394,  0.4869, -0.1476],
        [-0.4459, -0.0336,  0.0221, -0.0550]])

Alternatively, one can do the following

In [5]:
rvn2 = torch.normal(mean=0,std=1,size=(4,4)) # here the normal distribution is used
rvn2

tensor([[-0.9645,  0.0285, -0.3170,  1.6640],
        [ 0.7148,  0.3590, -0.1242,  2.0345],
        [ 0.9017, -1.1558,  0.1841,  0.0934],
        [ 0.3168, -0.8889,  1.1768,  0.8074]])

One can also specified the mean and std to using tensors **of same size**, in this case, the function can generate an output tensor that has the same size as the input mean and std tensor, each elment in the output tensor is generated as per the element specified in the mean tensor and std tensor correspondinly.

In [16]:
my_mean = [1., 2.]
my_std = [.2, .5]
mean = torch.tensor(my_mean) # two different means
std = torch.tensor(my_std) # two different std
rvn3 = torch.normal(mean,std) # here the normal distribution is used
rvn3 # two random numbers

tensor([0.6975, 2.3782])

Note that in the above generated tensor, the first element is generated from $\mathcal{N}(1.0,\sqrt{0.2})$, and the second one is from $\mathcal{N}(2.0,\sqrt{0.5})$ 

##### Uniform and Bernoulli distribution
Now, let's first draw numbers from a unifrom distribution in $[0,1]$, then use these randomly generated numbers to denote the probabilities that we gonna use for obtaining random numbers from a Bernoulli distribution

In [17]:
ud_from = 0
ud_to = 1
rvu = torch.Tensor(4,4).uniform_(ud_from,ud_to) # here the uniform distribution is used
rvu

tensor([[0.6167, 0.4009, 0.2426, 0.0523],
        [0.4494, 0.3270, 0.1688, 0.5733],
        [0.3116, 0.2653, 0.6158, 0.8620],
        [0.3405, 0.7099, 0.0871, 0.5649]])

In [7]:
rvbern = torch.bernoulli(rvu) # here the bernoulli distribution is used
rvbern

tensor([[0., 0., 1., 1.],
        [1., 1., 1., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.]])

##### Multinomial distribution
When using `torch.multinomial(P, num_samples, replacement)` to generate random numbers, one should first use a **non-negative** tensor $P$ to specify the probabilities or weights for each of the events, there must be **at least one non-zero element** in $P$:
* in case $P$ is a vector, say $P=[p_1,\dots,p_N]$, then each element $p_i,i=1\dots,N$ denotes the probabilities or weights for event $i$: if $P$ sums up to one, then $p_i$ denotes the related probabilites, if not, then elements of $P$ will be treated as weights (then being normalized to denote the prabilities). Input `num_samples` **must not** be larger than $N$ if `replacement=false` which is default when left blank.
* in case $P$ is a matrix, then each row of $P$ will be used correspondingly as described above. If $P$ has $m$ rows, then a tensor of $m$-by-`num_samples` will be generated as the output.

In [36]:
torch.manual_seed(myseed)
P1 = torch.randperm(11).float()
print('Below is the 10 probabilities/weights we gonna use for bernoulli distribution')
print(P1)

Below is the 10 probabilities/weights we gonna use for bernoulli distribution
tensor([ 4.,  5.,  9.,  2.,  8.,  1.,  0.,  3.,  6., 10.,  7.])


In [37]:
rvb_1 = torch.multinomial(P1,3)
rvb_1

tensor([0, 1, 7])

In [38]:
P2 = torch.tensor([0, 0.1, 0.5, 0.3, 0, 0.1])
rvb_2 = torch.multinomial(P2,3)
rvb_2

tensor([2, 3, 1])